# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids 
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


### Cleaning the data since scaler.fit function needs numeric data to work

In [5]:
# Replace not numerical data in the homeowner column
clean_df = pd.get_dummies(df, columns=['homeowner'])

# Checking for any null or non numeric feature values
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   loan_size           77536 non-null  float64
 1   interest_rate       77536 non-null  float64
 2   borrower_income     77536 non-null  int64  
 3   debt_to_income      77536 non-null  float64
 4   num_of_accounts     77536 non-null  int64  
 5   derogatory_marks    77536 non-null  int64  
 6   total_debt          77536 non-null  int64  
 7   loan_status         77536 non-null  object 
 8   homeowner_mortgage  77536 non-null  uint8  
 9   homeowner_own       77536 non-null  uint8  
 10  homeowner_rent      77536 non-null  uint8  
dtypes: float64(3), int64(4), object(1), uint8(3)
memory usage: 5.0+ MB


# Split the Data into Training and Testing

In [6]:
# Create our features
X = clean_df.copy()

# Dropping homeowner and loan_statis columns
X.drop(["loan_status"], axis=1, inplace=True)

# Create our target
y = clean_df["loan_status"]

In [7]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [9]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=1, stratify=y)

In [10]:
print(X_train)

       loan_size  interest_rate  borrower_income  debt_to_income  \
25729    10100.0          7.402            50300        0.403579   
20176     9200.0          7.049            46900        0.360341   
55080     8800.0          6.870            45200        0.336283   
10410     8900.0          6.919            45700        0.343545   
63110     8500.0          6.731            43900        0.316629   
...          ...            ...              ...             ...   
4767     12300.0          8.346            59100        0.492386   
26012     8300.0          6.634            43000        0.302326   
58637     6700.0          5.988            36900        0.186992   
23339     9200.0          7.054            47000        0.361702   
45677     7600.0          6.352            40400        0.257426   

       num_of_accounts  derogatory_marks  total_debt  homeowner_mortgage  \
25729                4                 1       20300                   1   
20176                3         

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [12]:
print(X_train)

       loan_size  interest_rate  borrower_income  debt_to_income  \
40824     8700.0          6.823            44800        0.330357   
35660     9700.0          7.249            48800        0.385246   
72265     8800.0          6.874            45300        0.337748   
20901     8300.0          6.659            43300        0.307159   
56733     7600.0          6.367            40500        0.259259   
...          ...            ...              ...             ...   
10937     8800.0          6.858            45100        0.334812   
18964     9400.0          7.139            47800        0.372385   
53377    11300.0          7.948            55400        0.458484   
33368     8700.0          6.843            45000        0.333333   
2576      8700.0          6.822            44800        0.330357   

       num_of_accounts  derogatory_marks  total_debt  homeowner_mortgage  \
40824                3                 0       14800                   1   
35660                4         

In [13]:
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
40824,8700.0,6.823,44800,0.330357,3,0,14800,1,0,0
35660,9700.0,7.249,48800,0.385246,4,0,18800,1,0,0
72265,8800.0,6.874,45300,0.337748,3,0,15300,1,0,0
20901,8300.0,6.659,43300,0.307159,2,0,13300,1,0,0
56733,7600.0,6.367,40500,0.259259,2,0,10500,0,1,0
...,...,...,...,...,...,...,...,...,...,...
10937,8800.0,6.858,45100,0.334812,3,0,15100,0,1,0
18964,9400.0,7.139,47800,0.372385,4,0,17800,0,0,1
53377,11300.0,7.948,55400,0.458484,5,1,25400,1,0,0
33368,8700.0,6.843,45000,0.333333,3,0,15000,0,0,1


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
simple_model = LogisticRegression(solver='lbfgs', random_state=1)
simple_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_simple = simple_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_simple)

0.9516305810328975

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_simple)

array([[  552,    56],
       [   87, 18689]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_simple))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       608
   low_risk       1.00      1.00      0.91      1.00      0.95      0.91     18776

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [21]:
# Resample the training data with the RandomOversampler
random_over_sampler = RandomOverSampler(random_state=1)
X_resampled, y_resampled = random_over_sampler.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56260, 'high_risk': 56260})

In [22]:
# Train the Logistic Regression model using the resampled data
naive_logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)
naive_logistic_regression_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred_naive = naive_logistic_regression_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_naive)

0.9956390296683336

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_naive)

array([[  606,     2],
       [  102, 18674]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_naive, digits=16))

                        pre       rec       spe        f1       geo       iba       sup

       high_risk  0.8559322033898306 0.9967105263157895 0.9945675330208777 0.9209726443768996 0.9956384531012427 0.9915083633468104       608
        low_risk  0.9998929106875134 0.9945675330208777 0.9967105263157895 0.9972231122503471 0.9956384531012427 0.9910834952408604     18776

     avg / total  0.9953774283290224 0.9946347503095336 0.9966433090271336 0.9948314343475895 0.9956384531012427 0.9910968216857849     19384



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
                           X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56260, 'high_risk': 56260})

In [27]:
# Train the Logistic Regression model using the resampled data
SMOTE_logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)
SMOTE_logistic_regression_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
y_pred_SMOTE = SMOTE_logistic_regression_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_SMOTE)

0.9956922891485211

In [29]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_SMOTE)

array([[  606,     2],
       [  100, 18676]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_SMOTE, digits=16))

                        pre       rec       spe        f1       geo       iba       sup

       high_risk  0.8583569405099151 0.9967105263157895 0.9946740519812527 0.9223744292237442 0.9956917685021270 0.9916039943556437       608
        low_risk  0.9998929221544063 0.9946740519812527 0.9967105263157895 0.9972766593688257 0.9956917685021270 0.9912002013701430     18776

     avg / total  0.9954534939228828 0.9947379281881964 0.9966466501088456 0.9949272703919267 0.9956917685021270 0.9912128667712565     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the data using the ClusterCentroids resampler
X_resampled, y_resampled = ClusterCentroids(random_state=1, sampling_strategy=1.0).fit_resample(
                            X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1892, 'low_risk': 1892})

In [32]:
# Train the Logistic Regression model using the resampled data
under_logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)
under_logistic_regression_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_pred_under = under_logistic_regression_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_under)

0.994233960487072

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_under)

array([[  604,     4],
       [   93, 18683]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_under, digits=16))

                        pre       rec       spe        f1       geo       iba       sup

       high_risk  0.8665710186513630 0.9934210526315790 0.9950468683425649 0.9256704980842914 0.9942336281612221 0.9883397954011077       608
        low_risk  0.9997859474500990 0.9950468683425649 0.9934210526315790 0.9974107786349199 0.9942336281612221 0.9886612193321467     18776

     avg / total  0.9956075179871589 0.9949958728848535 0.9934720480892905 0.9951605676065056 0.9942336281612222 0.9886511375249823     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the training data with SMOTEENN
X_resampled, y_resampled = SMOTEENN(random_state=1, sampling_strategy=1.0).fit_resample(
                            X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55612, 'low_risk': 55883})

In [37]:
# Train the Logistic Regression model using the resampled data
combination_model = LogisticRegression(solver='lbfgs', random_state=1)
combination_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [38]:
# Calculate the balanced accuracy score
y_pred_combination = combination_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_combination)

0.9956656594084273

In [39]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_combination)

array([[  606,     2],
       [  101, 18675]], dtype=int64)

In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_combination, digits=16))

                        pre       rec       spe        f1       geo       iba       sup

       high_risk  0.8571428571428571 0.9967105263157895 0.9946207925010652 0.9216730038022815 0.9956651111585483 0.9915561791339512       608
        low_risk  0.9998929164212668 0.9946207925010652 0.9967105263157895 0.9972498865244439 0.9956651111585483 0.9911418480227774     18776

     avg / total  0.9954154073394843 0.9946863392488651 0.9966449795679897 0.9948793363441367 0.9956651111585485 0.9911548439635323     19384



# Final Questions

1. Which model had the best balanced accuracy score?

    * 0.9516305810328975 - Simple Logistic Regression
    * 0.9956390296683336 - Naive Random Oversampling
    * **0.9956922891485211 - SMOTE Oversampling**
    * 0.994233960487072  - Undersampling
    * 0.9956656594084273 - Combination (Over and Under) Sampling<br><br>
    
    <u>**Conclusion:**</u>  The **SMOTE Oversampling** has the best balanced accuracy scores based on the values above. But also, the Combination (Over and Under) Sampling is very closed to the best.<br><br>

2. Which model had the best recall score?

    * 0.99			   - Simple Logistic Regression
    * 0.9946347503095336 - Naive Random Oversampling
    * 0.9947379281881964 - SMOTE Oversampling
    * **0.9949958728848535 - Undersampling**
    * 0.9946863392488651 - Combination (Over and Under) Sampling<br><br>
    
    <u>**Conclusion:**</u> The **Undersampling model** have the best recall score based on the values above.<br><br>
    
3. Which model had the best geometric mean score?

    * 0.95			     - Simple Logistic Regression
    * 0.9956384531012427 - Naive Random Oversampling
    * **0.9956917685021270 - SMOTE Oversampling**
    * 0.9942336281612222 - Undersampling
    * 0.9956651111585485 - Combination (Over and Under) Sampling<br><br>
    
    <u>**Conclusion:**</u>   The above figures show that the **SMOTE Oversampling** has the best geometric scores even though the Combination (Over and Under) Sampling is very close to the best.


